### LLM Model Demo

In [2]:
# Installed LangChain 
#%pip install langchain langchain-community faiss-cpu langchain-chroma langchain-openai langchain_text_splitters langgraph beautifulsoup4 langchain-huggingface

In [1]:
import langchain
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
import os

from langchain.cache import InMemoryCache

In [2]:
os.environ["AZURE_OPENAI_API_KEY"] = "ce53c5fce80c4503927244333e40634c"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://wdc-chat-llm.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"
os.environ["OPENAI_API_VERSION"] = "2024-06-01"


In [3]:
llm = AzureChatOpenAI(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version = os.environ["OPENAI_API_VERSION"],
    temperature= 0.9
)
langchain.llm_cache = InMemoryCache()
parser = StrOutputParser()
#embeddings = HuggingFaceEmbeddings() #model="sentence-transformers/all-mpnet-base-v2"

In [4]:
messages = [
    SystemMessage(content="Translate the following from English into Bengali"),
    HumanMessage(content="hi!"),
]

In [5]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [6]:
""" result = llm.invoke(messages)
parser.invoke(result) """

' result = llm.invoke(messages)\nparser.invoke(result) '

In [7]:
chain = prompt_template | llm | parser
chain.invoke({"language": "bengali", "text": "how are you doing"})

'তুমি কেমন আছ?'